# Object Relational Tutorial
https://docs.sqlalchemy.org/en/13/orm/tutorial.html  

In [67]:
%pylab inline

from pprint import pprint

Populating the interactive namespace from numpy and matplotlib


## Version check

In [68]:
import sqlalchemy
sqlalchemy.__version__

'1.3.3'

## Connecting

In [69]:
from sqlalchemy import create_engine
# engine = create_engine('sqlite:///:memory:', echo = True)
engine = create_engine('sqlite:///:memory:', echo = False)
engine

Engine(sqlite:///:memory:)

## Declare a Mapping

In [70]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
Base

sqlalchemy.ext.declarative.api.Base

In [71]:
from sqlalchemy import Column, Integer, String, Sequence

class User(Base):
    __tablename__ = 'users'
    
    id = Column(Integer, Sequence('user_id_seq'), primary_key=True)
    name = Column(String(50))
    fullname = Column(String(50))
    nickname = Column(String(50))
    
    def __repr__(self):
        return '<User(name = {}, fullname = {}, nickname = {})>'.format(self.name, self.fullname, self.nickname)

## Create a Schema

In [72]:
User.__table__

Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False, default=Sequence('user_id_seq', metadata=MetaData(bind=None))), Column('name', String(length=50), table=<users>), Column('fullname', String(length=50), table=<users>), Column('nickname', String(length=50), table=<users>), schema=None)

In [73]:
Base.metadata

MetaData(bind=None)

In [74]:
Base.metadata.create_all(engine)

## Create an Instance of the Mapped Class

In [75]:
ed_user = User(name = 'ed', fullname = 'Ed Jones', nickname = 'edsnickname')
ed_user

<User(name = ed, fullname = Ed Jones, nickname = edsnickname)>

In [76]:
ed_user.name

'ed'

In [77]:
str(ed_user.id)

'None'

## Creating a Session

In [78]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind = engine)
Session

sessionmaker(class_='Session', bind=Engine(sqlite:///:memory:), autoflush=True, autocommit=False, expire_on_commit=True)

In [79]:
Session = sessionmaker()
Session.configure(bind = engine)
Session

sessionmaker(class_='Session', bind=Engine(sqlite:///:memory:), autoflush=True, autocommit=False, expire_on_commit=True)

In [80]:
session = Session()
session

## Adding and Updating Objects

In [81]:
ed_user = User(name='ed', fullname='Ed Jones', nickname='edsnickname')
session.add(ed_user)
ed_user

<User(name = ed, fullname = Ed Jones, nickname = edsnickname)>

In [82]:
str(ed_user.id)

'None'

In [83]:
our_user = session.query(User).filter_by(name = 'ed').first()
our_user

<User(name = ed, fullname = Ed Jones, nickname = edsnickname)>

In [84]:
ed_user.id

1

In [85]:
our_user is ed_user

True

In [86]:
session.add_all([User(name='wendy', fullname='Wendy Williams', nickname='windy'),
                 User(name='mary', fullname='Mary Contrary', nickname='mary'),
                 User(name='fred', fullname='Fred Flintstone', nickname='freddy')])

In [87]:
ed_user.nickname = 'eddie'

In [88]:
session.dirty

IdentitySet([<User(name = ed, fullname = Ed Jones, nickname = eddie)>])

In [89]:
session.new

IdentitySet([<User(name = wendy, fullname = Wendy Williams, nickname = windy)>, <User(name = mary, fullname = Mary Contrary, nickname = mary)>, <User(name = fred, fullname = Fred Flintstone, nickname = freddy)>])

In [90]:
session.commit()

In [91]:
ed_user.id

1

## Rolling Back

In [92]:
ed_user.name = 'Edwardo'

In [93]:
fake_user = User(name='fakeuser', fullname='Invalid', nickname='12345')
session.add(fake_user)

In [94]:
session.query(User).filter(User.name.in_(['Edwardo', 'fakeuser'])).all()

[<User(name = Edwardo, fullname = Ed Jones, nickname = eddie)>,
 <User(name = fakeuser, fullname = Invalid, nickname = 12345)>]

In [95]:
ed_user.name

'Edwardo'

In [96]:
session.rollback()

In [97]:
ed_user.name

'ed'

In [98]:
fake_user in session

False

In [99]:
session.query(User).filter(User.name.in_(['Edwardo', 'fakeuser'])).all()

[]

## Querying

In [100]:
for instance in session.query(User).order_by(User.id):
    print(instance.name, instance.fullname)

ed Ed Jones
wendy Wendy Williams
mary Mary Contrary
fred Fred Flintstone


In [102]:
for name, fullname in session.query(User.name, User.fullname):
    print(name, fullname)

ed Ed Jones
wendy Wendy Williams
mary Mary Contrary
fred Fred Flintstone


In [103]:
for row in session.query(User, User.name).all():
    print(row.User, row.name)

<User(name = ed, fullname = Ed Jones, nickname = eddie)> ed
<User(name = wendy, fullname = Wendy Williams, nickname = windy)> wendy
<User(name = mary, fullname = Mary Contrary, nickname = mary)> mary
<User(name = fred, fullname = Fred Flintstone, nickname = freddy)> fred


In [104]:
for row in session.query(User.name.label('name_label')).all():
    print(row.name_label)

ed
wendy
mary
fred


In [106]:
from sqlalchemy.orm import aliased

user_alias = aliased(User, name='user_alias')

In [107]:
for row in session.query(user_alias, user_alias.name).all():
    print(row.user_alias)

<User(name = ed, fullname = Ed Jones, nickname = eddie)>
<User(name = wendy, fullname = Wendy Williams, nickname = windy)>
<User(name = mary, fullname = Mary Contrary, nickname = mary)>
<User(name = fred, fullname = Fred Flintstone, nickname = freddy)>


In [109]:
row.user_alias

<User(name = fred, fullname = Fred Flintstone, nickname = freddy)>

In [110]:
for u in session.query(User).order_by(User.id)[1:3]:
    print(u)

<User(name = wendy, fullname = Wendy Williams, nickname = windy)>
<User(name = mary, fullname = Mary Contrary, nickname = mary)>


In [111]:
for name, in session.query(User.name).filter_by(fullname ='Ed Jones'):
    print(name)

ed


In [112]:
for name, in session.query(User.name).filter(User.fullname=='Ed Jones'):
    print(name)

ed


In [113]:
for user in session.query(User).\
    filter(User.name=='ed').\
    filter(User.fullname=='Ed Jones'):
    print(user)

<User(name = ed, fullname = Ed Jones, nickname = eddie)>


## Common Filter Operators

In [117]:
list(session.query(User).filter(User.name == 'ed'))

[<User(name = ed, fullname = Ed Jones, nickname = eddie)>]

In [118]:
list(session.query(User).filter(User.name != 'ed'))

[<User(name = wendy, fullname = Wendy Williams, nickname = windy)>,
 <User(name = mary, fullname = Mary Contrary, nickname = mary)>,
 <User(name = fred, fullname = Fred Flintstone, nickname = freddy)>]

In [119]:
list(session.query(User).filter(User.name.like('%ed%')))

[<User(name = ed, fullname = Ed Jones, nickname = eddie)>,
 <User(name = fred, fullname = Fred Flintstone, nickname = freddy)>]

In [120]:
list(session.query(User).filter(User.name.ilike('%ed%')))

[<User(name = ed, fullname = Ed Jones, nickname = eddie)>,
 <User(name = fred, fullname = Fred Flintstone, nickname = freddy)>]

In [121]:
list(session.query(User).filter(User.name.in_(['ed', 'wendy', 'jack'])))

[<User(name = ed, fullname = Ed Jones, nickname = eddie)>,
 <User(name = wendy, fullname = Wendy Williams, nickname = windy)>]

In [124]:
list(session.query(User).filter(User.name.in_(
    session.query(User.name).filter(User.name.like('%ed%'))
)))

[<User(name = ed, fullname = Ed Jones, nickname = eddie)>,
 <User(name = fred, fullname = Fred Flintstone, nickname = freddy)>]

In [125]:
list(session.query(User).filter(~User.name.in_(['ed', 'wendy', 'jack'])))

[<User(name = mary, fullname = Mary Contrary, nickname = mary)>,
 <User(name = fred, fullname = Fred Flintstone, nickname = freddy)>]

In [126]:
list(session.query(User).filter(User.name == None))

[]

In [127]:
list(session.query(User).filter(User.name.is_(None)))

[]

In [128]:
list(session.query(User).filter(User.name != None))

[<User(name = ed, fullname = Ed Jones, nickname = eddie)>,
 <User(name = wendy, fullname = Wendy Williams, nickname = windy)>,
 <User(name = mary, fullname = Mary Contrary, nickname = mary)>,
 <User(name = fred, fullname = Fred Flintstone, nickname = freddy)>]

In [129]:
list(session.query(User).filter(User.name.isnot(None)))

[<User(name = ed, fullname = Ed Jones, nickname = eddie)>,
 <User(name = wendy, fullname = Wendy Williams, nickname = windy)>,
 <User(name = mary, fullname = Mary Contrary, nickname = mary)>,
 <User(name = fred, fullname = Fred Flintstone, nickname = freddy)>]

In [137]:
from sqlalchemy import and_

list(session.query(User).filter(and_(User.name == 'ed', 
                                     User.fullname == 'Ed Jones')))

[<User(name = ed, fullname = Ed Jones, nickname = eddie)>]

In [135]:
list(session.query(User).filter(User.name == 'ed', 
                                User.fullname == 'Ed Jones'))

[<User(name = ed, fullname = Ed Jones, nickname = eddie)>]

In [134]:
list(session.query(User).\
     filter(User.name == 'ed').\
     filter(User.fullname == 'Ed Jones'))

[<User(name = ed, fullname = Ed Jones, nickname = eddie)>]

In [141]:
from sqlalchemy import or_

list(session.query(User).filter(or_(User.name == 'ed', 
                                    User.name == 'wendy')))

[<User(name = ed, fullname = Ed Jones, nickname = eddie)>,
 <User(name = wendy, fullname = Wendy Williams, nickname = windy)>]

In [144]:
# list(session.query(User).filter(User.name.match('wendy')))

## Returning Lists and Scalars